# Generacion de Datasets utilizando RLHF para fine tunning

## Procedimiento

- Recoleccion de Datos: Obtener Informacion mediante documentos de textos seleccionables (no escaneados), mediante web, redes sociales, scrapping,etc
- Extraer Texto: Usa PyMuPDF para extraer texto de PDFs no escaneados, procesando en paralelo con ProcessPoolExecutor para manejar grandes volúmenes (>100, >10,000).
- Limpiar Datos: Elimina espacios múltiples y caracteres no deseados con expresiones regulares, asegurando texto coherente.
- Validar: Verifica que los fragmentos extraídos no estén vacíos y tengan una longitud mínima (por ejemplo, 50 caracteres).
- Generar Conversaciones: Divide el texto en párrafos, usa un modelo como meta-llama/Llama-3.2-1B-Instruct para generar diálogos conversacionales específicos al contenido, con al menos 4 intercambios por conversación, en formato {"messages": [{"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}], "topic": "..."}.
- Estructurar Dataset: Guarda las conversaciones en archivos JSONL con dos columnas: "messages" y "topic".
Cargar y Subir: Carga el dataset con load_dataset("json", data_files="path/*.jsonl") y súbelo a un repositorio privado en Hugging Face Hub con push_to_hub("username/dataset_name", private=True).

In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
%pip install --upgrade pymupdf nltk ollama


   ---------------------------------------- 0.0/18.7 MB ? eta -:--:--
   - -------------------------------------- 0.8/18.7 MB 4.8 MB/s eta 0:00:04
   ------ --------------------------------- 2.9/18.7 MB 8.4 MB/s eta 0:00:02
   ----------- ---------------------------- 5.5/18.7 MB 9.6 MB/s eta 0:00:02
   ----------------- ---------------------- 8.1/18.7 MB 10.5 MB/s eta 0:00:02
   --------------------- ------------------ 10.0/18.7 MB 10.2 MB/s eta 0:00:01
   ------------------------- -------------- 12.1/18.7 MB 10.2 MB/s eta 0:00:01
   ------------------------------ --------- 14.4/18.7 MB 10.3 MB/s eta 0:00:01
   ----------------------------------- ---- 16.5/18.7 MB 10.3 MB/s eta 0:00:01
   ---------------------------------------  18.6/18.7 MB 10.4 MB/s eta 0:00:01
   ---------------------------------------- 18.7/18.7 MB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 11.4 MB/s eta 0:00:00
 


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os

MIN_FRAGMENT_LENGTH = 500  # Longitud mínima de un fragmento
MAX_FRAGMENT_LENGTH = 2000  # Longitud máxima de un fragmento
REPEAT_THRESHOLD = 0.3  # Umbral para considerar un bloque como repetitivo

OLLAMA_MODEL = "llama3.1:8b"
TEMPERATURE = 0.7  # Equilibrio entre creatividad y coherencia
TOP_P = 0.9  # Filtrado de núcleo para diversidad
MIN_CONVERSATION_LENGTH = 3 

### Recoleccion de Datos

In [3]:
from pathlib import Path

# Verificar si la carpeta existe

folder_url = "./docs"
folder = Path(folder_url)


if folder.exists() and folder.is_dir():
    print("Valid Folder")
    
# Obtener todos los archivos de la carpeta
files = [f for f in folder.rglob("*") if f.is_file()]

files
len(files)

Valid Folder


89

### Extraccion de Texto y Limpieza de Datos

In [4]:
import pymupdf
import re
import os
import hashlib
from collections import Counter

# Constantes
MIN_FRAGMENT_LENGTH = 500
MAX_FRAGMENT_LENGTH = 2000
REPEAT_THRESHOLD = 0.3
MIN_BLOCK_LENGTH = 30  # Reducido de 50 a 30 para incluir bloques más cortos

def clean_text(text, repeated_blocks=None):
    """Limpia el texto, eliminando caracteres repetitivos, texto redundante y caracteres no deseados."""
    # Eliminar caracteres de control no deseados (excepto \n)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
    # Eliminar patrones repetitivos como ----, ...., ****
    text = re.sub(r'([^\w\s])\1{2,}|\s*[.]{3,}\s*', '', text)
    # Normalizar espacios múltiples
    text = re.sub(r'[ \t]+', ' ', text)
    # Eliminar espacios al inicio y final de cada línea
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    # Filtrar líneas duplicadas, números, correos y contenido administrativo
    seen_lines = set()
    unique_lines = []
    for line in lines:
        if line not in seen_lines and \
           not re.match(r'^\d+$', line) and \
           not re.match(r'.*@(.*\.)+.*', line) and \
           not re.match(r'^(Tel|Fax|E-mail|www\.).*', line, re.IGNORECASE) and \
           (repeated_blocks is None or line not in repeated_blocks):
            unique_lines.append(line)
            seen_lines.add(line)
    return '\n'.join(unique_lines)

def extract_text_from_pdf(pdf_path):
    """
    Extrae texto de un PDF en una sola pasada, preservando el texto del inicio de la página,
    eliminando encabezados, pies de página y contenido irrelevante,
    dividiendo en fragmentos de 500 a 2000 caracteres con metadata corregida.
    """
    try:
        doc = pymupdf.open(pdf_path)
        total_pages = len(doc)
        chunks = []
        current_chunk = []  # Lista de (párrafo, página)
        current_chunk_length = 0
        filename = os.path.basename(pdf_path)
        block_counter = Counter()

        for page_number in range(1, total_pages + 1):
            page = doc[page_number - 1]
            page_height = page.rect.height
            blocks = page.get_text("blocks")
            page_text = []

            # Procesar bloques y filtrar encabezados/pies de página
            for block in blocks:
                text = block[4]
                y0, y1 = block[1], block[3]
                # Excluir pies de página (parte inferior de la página)
                if y1 > 0.95 * page_height:  # Relajado de 0.95 a 0.9
                    continue
                # Excluir encabezados solo si son repetitivos
                if y0 < 0.05 * page_height:  # Relajado de 0.05 a 0.1
                    block_counter[text] += 1
                    if block_counter[text] > total_pages * REPEAT_THRESHOLD:
                        continue
                if text and len(text) >= MIN_BLOCK_LENGTH:
                    block_counter[text] += 1
                    page_text.append((text, page_number))

            # Si no hay texto válido en la página, continuar
            if not page_text:
                continue

            # Acumular párrafos con su número de página
            for paragraph, page in page_text:
                current_chunk.append((paragraph, page))
                current_chunk_length += len(paragraph) + 2  # +2 por "\n\n"

                # Si el fragmento alcanza la longitud mínima, procesarlo
                if current_chunk_length >= MIN_FRAGMENT_LENGTH:
                    chunk_text = "\n\n".join(p for p, _ in current_chunk)
                    cleaned_chunk = clean_text(chunk_text, None)
                    cleaned_length = len(cleaned_chunk)

                    # Obtener el rango de páginas del fragmento
                    page_numbers = sorted(set(page for _, page in current_chunk))
                    start_page = page_numbers[0] if page_numbers else page_number
                    end_page = page_numbers[-1] if page_numbers else page_number

                    # Dividir fragmentos largos
                    while cleaned_length > MAX_FRAGMENT_LENGTH:
                        sub_chunk = cleaned_chunk[:MAX_FRAGMENT_LENGTH]
                        last_paragraph_end = sub_chunk.rfind("\n\n")
                        if last_paragraph_end == -1:
                            last_paragraph_end = MAX_FRAGMENT_LENGTH
                        chunk_to_add = cleaned_chunk[:last_paragraph_end].strip()

                        # Calcular el número de páginas para el subfragmento
                        chars_so_far = 0
                        sub_chunk_pages = []
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far <= last_paragraph_end:
                                sub_chunk_pages.append(page)
                            else:
                                break
                        sub_start_page = min(sub_chunk_pages) if sub_chunk_pages else page_number
                        sub_end_page = max(sub_chunk_pages) if sub_chunk_pages else page_number

                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {len(chunk_to_add)} | "
                            f"PAGES: {sub_start_page}-{sub_end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + chunk_to_add, hashlib.md5(chunk_to_add.encode()).hexdigest()))
                        cleaned_chunk = cleaned_chunk[last_paragraph_end:].strip()
                        cleaned_length = len(cleaned_chunk)

                        # Actualizar current_chunk para los párrafos restantes
                        remaining_chunk = []
                        chars_so_far = 0
                        for paragraph, page in current_chunk:
                            chars_so_far += len(paragraph) + 2
                            if chars_so_far > last_paragraph_end:
                                remaining_chunk.append((paragraph, page))
                        current_chunk = remaining_chunk
                        current_chunk_length = cleaned_length
                        page_numbers = sorted(set(page for _, page in current_chunk))
                        start_page = page_numbers[0] if page_numbers else page_number

                    # Añadir el fragmento completo
                    if cleaned_length >= MIN_FRAGMENT_LENGTH:
                        metadata = (
                            f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                            f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                        )
                        chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))
                        current_chunk = []
                        current_chunk_length = 0

                    else:
                        current_chunk = [(cleaned_chunk, page_numbers[-1])] if page_numbers else []
                        current_chunk_length = cleaned_length

        # Añadir el fragmento final si cumple con la longitud mínima
        if current_chunk and current_chunk_length >= MIN_FRAGMENT_LENGTH:
            chunk_text = "\n\n".join(p for p, _ in current_chunk)
            cleaned_chunk = clean_text(chunk_text, None)
            cleaned_length = len(cleaned_chunk)
            if cleaned_length >= MIN_FRAGMENT_LENGTH:
                page_numbers = sorted(set(page for _, page in current_chunk))
                start_page = page_numbers[0] if page_numbers else total_pages
                end_page = page_numbers[-1] if page_numbers else total_pages
                metadata = (
                    f"# FILENAME: {filename} | CHARACTERS: {cleaned_length} | "
                    f"PAGES: {start_page}-{end_page}/{total_pages}\n\n"
                )
                chunks.append((metadata + cleaned_chunk, hashlib.md5(cleaned_chunk.encode()).hexdigest()))

        doc.close()

        # Filtrar bloques repetitivos y duplicados
        repeated_blocks = {text for text, count in block_counter.items() if count > total_pages * REPEAT_THRESHOLD}
        final_chunks = []
        seen_hashes = set()

        for chunk, chunk_hash in chunks:
            chunk_text = '\n'.join(line for line in chunk.splitlines() if not line.startswith('#'))
            cleaned_chunk = clean_text(chunk_text, repeated_blocks)
            if len(cleaned_chunk) >= MIN_FRAGMENT_LENGTH and chunk_hash not in seen_hashes:
                # Actualizar la longitud en la metadata después de la limpieza final
                metadata_lines = chunk.splitlines()[0]
                metadata = re.sub(r'CHARACTERS: \d+', f'CHARACTERS: {len(cleaned_chunk)}', metadata_lines)
                final_chunks.append(f"{metadata}\n\n{cleaned_chunk}")
                seen_hashes.add(chunk_hash)

        return final_chunks if final_chunks else None
    except Exception as e:
        print(f"Error procesando {pdf_path}: {e}")
        return None

In [10]:
chunks=extract_text_from_pdf(files[0])

In [12]:
chunks

['# FILENAME: 9 Recomendaciones Especiales GAFI.pdf | CHARACTERS: 2000 | PAGES: 1-1/3\n\nFinancial Action Task Force on Money Laundering\nGrupo de Acción Financiera sobre el lavado de activos\nGroupe d’action financière sur le blanchiment de capitaux\nLAS 9 RECOMENDACIONES ESPECIALES\nCONTRA EL FINANCIAMIENTO DEL TERRORISMO\nGrupo de Acción Financiera de Sudamérica\nGrupo de Açâo Financeira da América do Sul\nI.\nRatificación y ejecución de los instrumentos de las NU\nTodos los países deben dar pasos inmediatos para ratificar y ejecutar plenamente la Convención\nInternacional de las Naciones Unidas de 1999 para la Eliminación del Financiamiento del\nTerrorismo.\nLos países deben también poner en práctica de forma inmediata las resoluciones de las Naciones\nUnidas relacionadas con la prevención y eliminación del financiamiento de actos terroristas,\nparticularmente la Resolución 1373 del Consejo de Seguridad de las Naciones Unidas.\nII.\nTipificación del financiamiento del terrorismo y 

### Generar Conversaciones

In [76]:
from ollama import AsyncClient
from pydantic import BaseModel, Field
from typing import List


# Definir el esquema Pydantic para la salida estructurada
class Message(BaseModel):
    role: str = Field(..., pattern="^(system|user|assistant)$")
    content: str

class Conversation(BaseModel):
    messages: List[Message] = Field(..., min_items=MIN_CONVERSATION_LENGTH)
    topic: str

ollama_client = AsyncClient()

async def generate_conversation(fragment, temperature=TEMPERATURE, top_p=TOP_P):
    """
    Genera una conversación estructurada en formato JSON usando Ollama, basada en un fragmento de texto.
    Optimizado para múltiples iteraciones en la generación de datasets para fine-tuning.
    """
    if not fragment or len(fragment) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Fragmento demasiado corto ({len(fragment)} caracteres).")
        return None

    # Extraer el texto sin la metadata (líneas que comienzan con '#')
    fragment_content = '\n'.join(line for line in fragment.splitlines() if not line.startswith('#')).strip()
    if len(fragment_content) < MIN_FRAGMENT_LENGTH:
        print(f"Error: Contenido útil del fragmento demasiado corto ({len(fragment_content)} caracteres).")
        return None

    # Prompt optimizado para múltiples iteraciones
    prompt = f"""
Basado en el siguiente contexto:
<context>
{fragment}
</context>

<instructions>
Genera una conversación entre un usuario y un asistente basada **exclusivamente** en el contenido de <context>...</context>. La conversación debe seguir este formato y estilo:

1. **Formato de salida**:
   - Devuelve un objeto JSON con dos campos:
     - "messages": lista de objetos con la forma {{ "role": "user" | "assistant", "content": "string" }}
     - "topic": cadena corta (máximo 50 caracteres) que resume el tema del fragmento.

2. **Estructura de la conversación**:
   - Longitud: entre **1 y 2 intercambios** (2 a 4 mensajes).
   - Usa UNA de estas dos opciones:
     - **Opción A**: 1 intercambio (2 mensajes). Ambos deben ser **largos y detallados**.
     - **Opción B**: 2 intercambios (4 mensajes). Cada respuesta debe ser extensa y cubrir bien el contenido.
   - cada intercambio (par de mensajes) debe cumplir lo siguiente: empieza con el role: "user" y termina con el role: "assistant", en otras palabras, formato conversacional cumpliento el formato SFT (Supervices Fine Tunning)

3. **Intervenciones del usuario**:
   - **No incluyas frases como**:
     - “según el texto”, “en el fragmento”, “respecto al documento”, “basado en este artículo”, etc.
   - Deben ser naturales, humanas y relevantes.
   - Pueden adoptar estas formas:
     - Preguntas específicas o inferenciales.
     - Frases incompletas para completar.
     - Comentarios o afirmaciones parcialmente desarrolladas.
   - Las intervenciones deben seguir un flujo lógico y coherente.

4. **Respuestas del asistente**:
   - Claras, profesionales y bien estructuradas.
   - Basadas **únicamente en el contenido del contexto**.
   - Extensas, ricas en detalles y sin añadir información inventada.
   - Usa un tono conversacional técnico o divulgativo según el tema.

5. **Cobertura y contenido**:
   - Cubre **todo el contenido** del fragmento.
   - No ignores secciones importantes ni temas mencionados.
   - Si el texto es poco claro, genera igualmente un solo intercambio lo más plausible posible.

6. **Objetivo de esta conversación**:
   - Este formato será usado para entrenar modelos mediante *Supervised Fine-Tuning (SFT)* usando `SFTTrainer` de Hugging Face.
   - Busca generar conversaciones verosímiles y útiles para entrenar modelos en comprensión lectora, generación coherente y natural.

<example>
**Fragmento técnico**:
PERIODO DE TRANSICIÓN RELATIVO A LA IMPLEMENTACIÓN GENERAL DEL ACUERDO  
215. El Nuevo Acuerdo será aplicable a todos los bancos internacionalmente activos en cada nivel del grupo bancario. Aquellos países en los que la subconsolidación no es actualmente un requisito, tendrán un periodo de transición de tres años, a partir de la fecha de implementación, para aplicarla.  

PERIODO DE TRANSICIÓN RELATIVO AL MÉTODO FUNDADO EN LA CALIFICACIÓN INTERNA  
216. El Comité reconoce que un respeto completo e inmediato de ciertos requisitos relacionados con datos podría no ser posible, particularmente para exposiciones frente a empresas, bancos, soberanos y retail. Para estos casos, se prevé un periodo de transición limitado durante el cual las entidades podrán utilizar aproximaciones alternativas.

**Salida esperada**:
```json
{{
  "messages": [
    {{
      "role": "user",
      "content": "¿Qué ocurre en los países donde la subconsolidación no es obligatoria y cómo afecta esto a la aplicación del acuerdo?"
    }},
    {{
      "role": "assistant",
      "content": "En esos países, se establece un periodo de transición de tres años a partir de la implementación del acuerdo. Esto les permite adaptarse gradualmente a los nuevos requisitos regulatorios sin enfrentar impactos abruptos. La subconsolidación es esencial para asegurar una visión completa del riesgo dentro de los grupos bancarios, y este periodo transitorio facilita su adopción progresiva."
    }},
    {{
      "role": "user",
      "content": "¿Qué flexibilidades se contemplan para las entidades financieras en relación con los datos requeridos por el método basado en calificación interna?"
    }},
    {{
      "role": "assistant",
      "content": "El texto reconoce que ciertos requisitos de datos pueden ser difíciles de cumplir de inmediato, especialmente en exposiciones a empresas, bancos, gobiernos y clientes minoristas. Por ello, se permite un periodo de transición limitado durante el cual las instituciones pueden aplicar aproximaciones alternativas. Esto proporciona margen para ajustar los sistemas internos y recopilar datos adecuados sin comprometer la supervisión del riesgo."
    }},
    {{
      "role": "user",
      "content":"acuerdo 215"
    }},
    {{
      "role": "assistant",
      "content":"El Nuevo Acuerdo se aplicará a todos los bancos con actividad internacional en cada nivel de su grupo bancario. Los países donde la subconsolidación no sea actualmente obligatoria dispondrán de un período de transición de tres años, a partir de la fecha de implementación, para adoptarla."
    }}
  ],
  "topic": "Transición en la implementación del acuerdo"
}}
```

**Fragmento simple**:
El cielo es azul porque las moléculas en la atmósfera dispersan la luz azul del sol en todas direcciones. Esta dispersión se llama dispersión de Rayleigh.

**Salida esperada**:
```json
{{
  "messages": [
    {{
      "role": "user",
      "content": "¿Por qué el cielo tiene ese color azul tan característico durante el día?"
    }},
    {{
      "role": "assistant",
      "content": "El color azul del cielo se debe a un fenómeno llamado dispersión de Rayleigh. Cuando la luz del sol entra en la atmósfera terrestre, interactúa con las moléculas de aire. La luz azul, que tiene una longitud de onda más corta, se dispersa mucho más fácilmente que otros colores. Esta dispersión hace que el azul se difunda en todas direcciones, lo cual es lo que vemos desde cualquier punto del planeta durante el día. Por eso el cielo parece predominantemente azul cuando el sol está alto."
    }}
  ],
  "topic": "Dispersión de la luz en la atmósfera"
}}
```
</example>
    """

    try:
        # Llama a Ollama con roles system y user
        response = await ollama_client.chat(
            model=OLLAMA_MODEL,
            messages=[
                {
                    "role": "system",
                    "content": "Eres un experto en el dominio del fragmento proporcionado (por ejemplo, regulaciones financieras, riesgos financieros, finanzas). Genera respuestas claras, concisas y basadas únicamente en el fragmento, manteniendo un tono profesional y conversacional, se especifico y detalla una respuestas en el intercambio de asistente."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            options={
                "temperature": temperature,
                "top_p": top_p,
            },
            format=Conversation.model_json_schema()  # Especifica el esquema JSON
        )
        conversation = Conversation.model_validate_json(response.message.content)
        # Validar que la conversación tenga al menos un intercambio completo
        if len(conversation.messages) < 2:
            print(f"Error: Conversación inválida, número de mensajes insuficiente: {len(conversation.messages)}")
            return None
        # Si el número de mensajes es impar, eliminar el último (asumiendo que es del usuario)
        if len(conversation.messages) % 2 != 0 and len(conversation.messages) > 2:
            conversation.messages = conversation.messages[:-1]
        return conversation
    except Exception as e:
        print(f"Error generando conversación con Ollama: {e}")
        return None

In [81]:
import json
import asyncio
async def process_pdf(index, pdf_path, output_dir):
    """Procesa un PDF, genera conversaciones y las guarda en JSONL."""
    try:
        
        pages_text = extract_text_from_pdf(pdf_path)

        if not pages_text:
            return
    
        jsonl_file = os.path.join(output_dir, f"pdf_{index:04d}.jsonl")
        with open(jsonl_file, "w", encoding="utf-8") as f:
            tasks = [generate_conversation(fragment) for fragment in pages_text if len(fragment) > 20]
    
            conversations = await asyncio.gather(*tasks,)
            if conversations:
                output = [messages.model_dump() for messages in conversations if messages is not None]
                output_str = "\n".join(json.dumps(messages, ensure_ascii=False) for messages in output)
                f.write(output_str + "\n")
    except Exception as e:
        print(f"Error el pdf {pdf_path}: {e}")
    

In [82]:
await process_pdf(0,files[0],"outputs")

In [17]:
os.cpu_count()

20

In [18]:
import multiprocessing
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
import asyncio
import os
from pathlib import Path

def process_pdf_wrapper(args):
    index, pdf_path, output_dir = args
    # Ejecutar la función asíncrona process_pdf dentro de un bucle de eventos
    asyncio.run(process_pdf(index, pdf_path, output_dir))

def generate():
    pdf_files = files
    output_dir="outputs"
    output_folder_path=Path(output_dir)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)
    
    num_workers = int(min(multiprocessing.cpu_count() * 0.8,len(pdf_files)))
    with ProcessPoolExecutor(max_workers=8) as executor:
        list(tqdm(executor.map(process_pdf_wrapper, [(i, p, output_dir) for i, p in enumerate(pdf_files)]), total=len(pdf_files)))

In [19]:
%%time
generate()

  0%|          | 0/89 [00:00<?, ?it/s]


BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

### Saving

In [11]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="./outputs/*.jsonl")

dataset


Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['messages', 'topic'],
        num_rows: 664
    })
})

In [10]:
from huggingface_hub import login
login()

ERROR! Session/line number was not unique in database. History logging moved to new session 16


In [20]:
dataset.push_to_hub("jeanmcm/b_risks")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########|  270kB /  270kB            

README.md:   0%|          | 0.00/376 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jeanmcm/b_risks/commit/13e6a6ad8f5d4eb1efc3bf36fe4e0b58c6e237fb', commit_message='Upload dataset', commit_description='', oid='13e6a6ad8f5d4eb1efc3bf36fe4e0b58c6e237fb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jeanmcm/b_risks', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jeanmcm/b_risks'), pr_revision=None, pr_num=None)

# Testing RAG with Open WebUI 

In [36]:
import requests
import json
import sys
import time
from IPython.display import display, clear_output, HTML

url = "https://vwlppjjfa98c9x-8080.proxy.runpod.net"
api_key ="sk-05568562f28844fe997cadf960a346cd"

messages =  [{"role": "user", "content": "Que es el Riesgo Financiero?"}]

try:
    # Realizar la solicitud con stream=True
    with requests.post(f"{url}/api/chat/completions", stream=True,headers={
      "Content-Type": "application/json",
      "Authorization": f"Bearer {api_key}"
    },json={
      "model":"bosft-riesgos-rag-model",
      "messages":messages,
      "stream":True
      }) as response:
        response.raise_for_status()
        # Variable para almacenar la salida acumulada
        accumulated_output = ""

        # Iterar sobre las líneas de la respuesta
        for line in response.iter_lines():
            if line:
                # Decodificar la línea
                decoded_line = line.decode('utf-8').strip()
                # Si la línea comienza con "data:", extraer el contenido
                if decoded_line.startswith("data:"):
                    decoded_line = decoded_line[5:].strip()  # Quitar "data: "

                # Ignorar líneas vacías o marcadores de fin como "[DONE]"
                if not decoded_line or decoded_line == "[DONE]" :
                    continue
                
                
                try:
                    # Parsear si es JSON
                    data = json.loads(decoded_line)
                    if "choices" not in data: continue
                    
                    delta = data['choices'][0]['delta']
                    if "content" in delta: new_data = delta['content']
                except json.JSONDecodeError:
                    # Si no es JSON, usar la línea como texto
                    new_data = decoded_line

                # Acumular y mostrar la salida dinámicamente
                if new_data:
                    accumulated_output += new_data
                    # Limpiar la salida anterior y mostrar la nueva
                    clear_output(wait=True)
                    display(HTML(f"<b>Respuesta en streaming:</b> {accumulated_output}"))
                    time.sleep(0.1)  # Pequeña pausa para visibilidad
                    

except requests.exceptions.RequestException as e:
    print(f"\nError en la solicitud: {e}")

# Testing with Flowise